# Utilisation de l'API Spotify

In [ ]:
# Listes des codes fonctionnels pour la gestion de l'API Spotify à travers l'utilisation de la bibliothèque Spotipy

- Importation des bibliothèques principales et implémentation des logs

In [1]:
# Kernel 3.11.9 pour mon VSC
# on importe les librairies
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time # on importe le time pour faire des pauses vis à vis de l'api qui nous donne un rate limit

# Premier compte : client_id='5a63812d960f4789a86499ecb6073348', client_secret='31cf73ebc10247fcb28175c1a777f34a'
# deuxième compte : client_id='f692473a0b79476398c28bc4c2248aec', client_secret='dcffa96d20e54c9c85016ffc5bc1fcdb'

# on créé une connexion avec les données du compte.
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='5a63812d960f4789a86499ecb6073348',

                                               client_secret='31cf73ebc10247fcb28175c1a777f34a',

                                               redirect_uri='http://localhost:8080',

                                               scope='user-library-read'))

### Récupération de la liste des artistes et groupes

In [ ]:
# on utilise un dataframe de deux colonnes pour récupérer nom et id des artistes
df_list_artist = pd.DataFrame(columns=['artist_name','artist_id'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_artist_year( year):    
    
    # nous avons besoin de deux listes pour récupérer les noms et les id spotify que l'on réinitialise à chaque fois
    artist_name = []
    id_artist_spotify = []
    
    # on est limité à 1000 retours max que l'on fait à travers des pages de 50 retours via un for
    for i in range(0,1000,50):
        
        # on lance l'API pour récupérer les données de la page en cours avec l'année en paramètre (en str)
        track_results = sp.search(q='year:'+str(year), type='album', limit=50,offset=i)
        
        # on intègre les données récupérées dans les bonnes listes
        # on parcours les différents albums de la page
        for i, t in enumerate(track_results['albums']['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/search
            artist_name.append(t['artists'][0]['name'])
            id_artist_spotify.append(t['artists'][0]['id'])
    
    # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_artist = pd.DataFrame({'artist_name' : artist_name, 'artist_id' : id_artist_spotify})
    df_artist.drop_duplicates(inplace=True)
    df_list_artist2 = pd.concat([df_list_artist, df_artist])
    
    # on affiche un petit message pour chaque année récupérée
    print(f'Année {year} effectuée ! avec {df_artist.shape[0]} lignes sur l"année et {df_list_artist2.shape[0]} lignes sur la liste')
    return df_list_artist2

In [ ]:
# mise en place de la fonction sur une routine de 25 ans. Problème, nous sommes limités à 1000 retours par pages de 50 max. On passe peut être à coté de certains artistes
for year in range(2000, 2026, 1):
    df_list_artist = search_artist_year(year)
    # on retire les doublons puisque le même artistes peut parfaitement sortir plusieurs titres la même année et l'on souhaite juste la liste des artistes. 
    df_list_artist.drop_duplicates(inplace=True)

In [ ]:
# export du DF en CSV pour la conservation
df_list_artist.to_csv('liste_artistes.csv', index=False)

### Récupération de la liste des albums

In [2]:
# pour la mise en place nous avons utilisé un fichier créé grace à la fonction précédente.  
data_artistes = pd.read_csv('df_list_artist_p2.csv')

# vu que c'est le nouveau fichier, je ne veux que les id avec les noms
data_artistes = data_artistes[['artist_name','artist_id']]

In [3]:
# on reprends là où on s'était arreté (voir le fichier csv sauvegardé) pour en faire 2000 de plus. Il faut penser à reset l'index pour la fonction
data_artistes = data_artistes[5754:7754]
data_artistes.reset_index(inplace=True)

In [4]:
# récupération des données via l'api pour la partie Albums

# nous avons besoin de listes pour récupérer les infos des albums
df_list_album = pd.DataFrame(columns=['artist_id','album_name','album_id','release_date','total_tracks','url_image'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_album_artist(artist_id):        
    
    # nous avons besoin de listes pour récupérer les noms et les id spotify que l'on réinitialise à chaque fois
    album_name = []
    album_id = []
    release_date = []
    total_tracks = []
    url_image = []
    
    # on lance l'API pour récupérer les données souhaitées
    albums_results = sp.artist_albums(artist_id, include_groups='album')    
    
    # on intègre les données récupérées dans les bonnes listes
    # on parcours les différents albums de la page
    for i, t in enumerate(albums_results['items']):
        # pour chaque album on récupère les infos souhaitées
        # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/get-an-artists-albums

        album_name.append(t['name'])
        album_id.append(t['id'])
        release_date.append(t['release_date'])
        total_tracks.append(t['total_tracks'])
        try:
            url_image.append(t['images'][0]['url'])
        except:
            url_image.append("")


    # une fois les  lignes récupérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_album = pd.DataFrame({'artist_id':artist_id,\
                            'album_name' : album_name,\
                            'album_id' : album_id,\
                            'release_date' : release_date,\
                            'total_tracks' : total_tracks,\
                            'url_image' : url_image})
    
    df_list_album2 = pd.concat([df_list_album, df_album])

    time.sleep(1)

    # on retourne le df créé
    return df_list_album2

In [ ]:
# le test fonctionne bien pour un artiste
test = search_album_artist('1EH9eSje47IiRyVsq3gfkl')
test

In [6]:
# ici on récupère la totalité des lignes de notre fichier d'artiste avec un fichier de 12000 lignes c'est 3h. J'ai ajouté un print pour connaitre le nom de l'artiste ajouté
nb=0
for i in range (0,len(data_artistes)):
    df_list_album = search_album_artist(data_artistes.loc[i,'artist_id'])
    nb += 1
    print(f"Albums de {data_artistes.loc[i,'artist_name']} récupérés soit {nb} requètes sur {len(data_artistes)}")

Albums de The Postal Service récupérés soit 1 requètes sur 2000
Albums de Priscilla récupérés soit 2 requètes sur 2000
Albums de The RH Factor récupérés soit 3 requètes sur 2000
Albums de Jean-François Michaël récupérés soit 4 requètes sur 2000
Albums de Shinedown récupérés soit 5 requètes sur 2000
Albums de Reggae Gold récupérés soit 6 requètes sur 2000
Albums de Kaysha récupérés soit 7 requètes sur 2000
Albums de Il Divo récupérés soit 8 requètes sur 2000
Albums de Engin Nursani récupérés soit 9 requètes sur 2000
Albums de Mark Ronson récupérés soit 10 requètes sur 2000
Albums de Fischerspooner récupérés soit 11 requètes sur 2000
Albums de Soilwork récupérés soit 12 requètes sur 2000
Albums de Haifa Wehbe récupérés soit 13 requètes sur 2000
Albums de Elizio récupérés soit 14 requètes sur 2000
Albums de Petey Pablo récupérés soit 15 requètes sur 2000
Albums de Anny Versini récupérés soit 16 requètes sur 2000
Albums de Daddy Mory récupérés soit 17 requètes sur 2000
Albums de Broken Soc

Albums de Cascada récupérés soit 1003 requètes sur 2000


: 

: 

In [31]:
# export du df 
df_list_album.to_csv('liste_albums_artistes_3754-5754.csv', index=False)

### Récupération des informations des artistes

In [ ]:
# on récupère le fichier csv souhaité. Il doit contenir la colonne artist id avec les id des artistes recherchés.
data_artistes = pd.read_csv('liste_artistes.csv')

In [4]:
# récupération des données via l'api pour la partie Albums
# via cette API https://developer.spotify.com/documentation/web-api/reference/get-an-artist
# création d'une fonction récupérant le dataframe des artistes avec leurs id pour en récupérer les informations
# on utilise un dataframe en entrée qui doit contenir au moins deux colonnes. une avec les id artiste ('artist_id')
# et l'autre avec les nom d'artiste (artist_name) . on pourrait faire sans le second vu que le nom est dans l' API
# a voir si on le fait qu'avec une liste d'ID ?
def search_infos_artist(df):
    # nous avons besoin de listes pour récupérer les infos des artistes
    artist_name = []
    artist_id = []
    artist_popularity = []
    artist_followers = []
    artist_genres = []
    artist_image = []
    # on parcours la liste des id des artistes et grace au df on peut récupérer le nom.
    for ele, val in enumerate(df['artist_id']):
        # pour chaque artistes on lance une requête sur l'API pour récupérer les données
        artist_results = sp.artist(val)
        # on créé une liste des informations de l'liste_artistes
        artist_name.append(artist_results['name'])
        artist_id.append(val)
        artist_popularity.append(artist_results['popularity'])
        artist_followers.append(artist_results['followers']['total'])
        artist_genres.append(artist_results['genres'])
        try:
            artist_image.append(artist_results['images'][0]['url'])
        except:
            artist_image.append("")
        time.sleep(5)
        print(artist_results['name'], " ajouté")
    
    # une fois les lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_artist = pd.DataFrame({'artist_name':artist_name,\
                                'artist_id' : artist_id,\
                                'popularity' : artist_popularity,\
                                'artist_followers' : artist_followers,\
                                'genres' : artist_genres,\
                                'image' : artist_image})
    # on patiente 1 seconde entre deux DF pour limiter l'accès à l'API

    # on retourne le df créé
    return df_artist

In [ ]:
# test sur 1000 lignes de notre data. Le but sera de le faire une une seule fois pour les 11000 artiste. Soit 3h. 
df = search_infos_artist(data_artistes.head(1754))

In [ ]:
# on exporte le résultats en fichier CSV
df.to_csv('info_artist_0-1754.csv', index=False)

### Récupération de la liste des titres par albums

In [ ]:
# on conserve le df créé précédemment. ou on le récupère

df_list_album = pd.read_csv('liste_albums_artistes_0-1754.csv') 

In [ ]:
# création de liste de 20 éléments pour l'API des titres. On va faire une liste de liste

# on ne conserve que la colonne album_id pour créer une liste de ces id
liste_albums_global = list(df_list_album['album_id'])

# définition d'une fonction itérative utilisant le yield (Contrairement à return, yield ne termine pas la fonction ; 
# il suspend son exécution et reprend là où elle s’était arrêtée.)  cf : https://blog.alphorm.com/maitriser-yield-python

def creation_nb_liste(lst,nb):  # création d'une fonction qui prends deux arguments, une liste et un nombre. On utilisera ce nombre pour déterminer le nombre d'élément de la sous liste
    for ele in range(0,len(lst),nb): # on parcours la liste avec un pas du nombre nb
        yield lst[ele:ele + nb]  # a chaque itération on slicer la liste de nb valeurs pour créer une sous liste.

In [ ]:
# on utilise la fonction précédente pour générer des listes de 20 id. Nous aurons au final un liste de listes pour l'envoyer à l'API qui peut la gérer
liste_album_par_vingt = list(creation_nb_liste(liste_albums_global, 20))

In [ ]:
# récupération des données via l'api pour la partie titres

# nous avons besoin de listes pour récupérer les infos des albums
df_track_album = pd.DataFrame(columns=['album_name','album_id','album_type','total_tracks','release_date','release_date_precision','upc','label','popularity','images','disc_number','track_number','track_id','track_name','duration_ms'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_track_album(liste_albums):    
    
    # nous avons besoin de listes pour récupérer les informations
    
    album_name = []
    album_id = []
    album_type = []
    total_tracks = []
    release_date = []
    release_date_precision = []
    upc = []
    label = []
    popularity = []
    images = []
    disc_number = []
    track_number = []
    track_id = []
    track_name = []
    duration_ms = []
        
    # on lance l'API pour récupérer les données, pas de notion de pages ici. Nous avons des sous listes pour avoir tous les titres et nous fonctionnons avec 
    # une liste des albums recherchés.
    albums_track_results = sp.albums(liste_albums)

    # On parcours le nombre d'album que nous aurons en retour
    for ligne,valeur in enumerate(albums_track_results['albums']):
    # on intègre les données récupérées dans les bonnes listes, on utilise un enumérate pour lister les différents titres des albums. 
    # il y aura deux partie, une avec des append sur les infos globales toujours identiques et une partie sur les infos de chaque titre

        for i, t in enumerate(albums_track_results['albums'][ligne]['tracks']['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/get-multiple-albums

            # on récupère les quelques données propres à l'album sans utiliser la boucle mais on le fait dans la boucle pour avoir des tableaux de même valeurs pour le df:
            album_name.append(albums_track_results['albums'][ligne]['name'])
            album_id.append(albums_track_results['albums'][ligne]['id'])
            album_type.append(albums_track_results['albums'][ligne]['album_type'])
            total_tracks.append(albums_track_results['albums'][ligne]['total_tracks'])
            release_date.append(albums_track_results['albums'][ligne]['release_date'])
            release_date_precision.append(albums_track_results['albums'][ligne]['release_date_precision'])
            
            # mise en place d'un try puisque certaines valeurs sont manquantes et retourne une erreur. 
            try :
                upc.append(albums_track_results['albums'][ligne]['external_ids']['upc'])
            except:
                upc.append('')
            label.append(albums_track_results['albums'][ligne]['label'])
            popularity.append(albums_track_results['albums'][ligne]['popularity'])
            
            try:
                images.append(albums_track_results['albums'][ligne]['images'][0]['url'])
            except:
                images.append('')    
            # on récupère les données propre au titre en utilisant la boucle et ses valeurs
            disc_number.append(t['disc_number'])
            track_number.append(t['track_number'])
            track_id.append(t['id'])
            track_name.append(t['name'])
            duration_ms.append(t['duration_ms'])

        
    # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_track = pd.DataFrame({'album_name':album_name,
                            'album_id' : album_id,
                            'album_type' : album_type,
                            'total_tracks' : total_tracks,
                            'release_date' : release_date,
                            'release_date_precision' : release_date_precision,
                            'upc' : upc,
                            'label' : label,
                            'popularity' : popularity,
                            'images' : images,
                            'disc_number' : disc_number,
                            'track_number' : track_number,
                            'track_id' : track_id,
                            'track_name' : track_name,
                            'duration_ms' : duration_ms})
        
        
    df_list_track_album2 = pd.concat([df_track_album, df_track])
    
    # on retourne le df créé
    return df_list_track_album2

In [ ]:
# utilisation de la fonction via une boucle et un message pour nous indiquer l'action en cours. 
df = pd.DataFrame()
for ele in range (len(liste_album_par_vingt)):
    df = pd.concat([df,search_track_album(liste_album_par_vingt[ele])])
    print(f"Liste n°{ele} de vingt albums récupérée sur {len(liste_album_par_vingt)} listes demandées")

In [ ]:
# on conserve le résultat en fichier CSV (Attention certaines colonnes sont en liste, et donc seront transformées en string) Il faudra y penser pour l'utilisation
df.to_csv('liste_album_track_0-1754.csv', index=False)